In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json


In [2]:

path = 'E:\Technologie\Master\Webm\wem'

with open(f'{path}\\movies.json',encoding="utf8") as f:
    data = json.load(f)

with open(f'{path}\\nlp.json',encoding="utf8") as f:
    data_nlp = json.load(f)
 


In [3]:
df_nlp = pd.DataFrame(data_nlp).T
df_data = pd.DataFrame(data)

In [4]:
# set the uuid as index
df_data.set_index('uuid', inplace=True)

In [5]:
# join the two dataframes on the index and uuid
df = df_data.join(df_nlp, how="inner")

In [6]:
# add a column uuid from the index
df['uuid'] = df.index

In [13]:
# show where the signature is missing

df

,title,year,image,actors,plot,signature,ttr,fre,readability,tags,uuid,signature_missing
2327bfa4f30011edb451b8aeed79c0cc,Buffy the Vampire Slayer,1992,Buffy The Vampire Slayer Movie.jpg,"[Kristy Swanson, Luke Perry, Rutger Hauer, Pau...",Buffy Summers is a cheerleader at Hemery High ...,"[[0.4492585610019946, -0.10435049338123388, 0....",0.191702,98.62,2.0,"[action, journey, adventure]",2327bfa4f30011edb451b8aeed79c0cc,False
26c483fef30011edb451b8aeed79c0cc,Fearless,1993,Fearless ver1.jpg,"[ \n, Jeff Bridges, Isabella Rossellini, Rosie...",Max Klein survives an airline crash. The plane...,"[[0.36750598188921874, 0.028326033376961626, -...",0.154756,99.23,2.0,"[mystery, conspiracy, suspense]",26c483fef30011edb451b8aeed79c0cc,False
271c9396f30011edb451b8aeed79c0cc,Show Me Love,1998,Fucking Åmål original poster.jpg,"[Alexandra Dahlström, Rebecka Liljeberg, Eric...","Two girls, Agnes and Elin, attend school in th...","[[0.39478842674431, -0.017388587699599136, 0.0...",0.137574,99.23,3.0,"[business, action, journey]",271c9396f30011edb451b8aeed79c0cc,False
2b5896d0f30011edb451b8aeed79c0cc,Miss Congeniality,2000,Miss Congeniality Poster.jpg,"[ Sandra Bullock\n, Michael Caine, Benjamin Br...","In 1982, a young Gracie Hart steps into a play...","[[0.4135457351151562, 0.24593668651936273, -0....",0.203982,90.05,3.0,"[business, action, journey]",2b5896d0f30011edb451b8aeed79c0cc,False
2c134ed0f30011edb451b8aeed79c0cc,Mulholland Drive,2001,Mulholland.png,"[Naomi Watts, Justin Theroux, Laura Harring, L...",A dark-haired woman is the sole survivor of a ...,"[[0.31173353955770544, -0.013913081915504734, ...",0.170954,97.8,3.0,"[mystery, conspiracy, suspense]",2c134ed0f30011edb451b8aeed79c0cc,False
...,...,...,...,...,...,...,...,...,...,...,...,...
024118bcf31e11edb451b8aeed79c0cc,Thalasso,2019,Thalasso 2019 film poster.png,"[br, Gérard Depardieu]",The film is set at a centre for seawater thera...,"[[0.18531734022405608, 0.5044102884069372, -0....",0.177334,98.72,5.0,"[celebrity, action, crime]",024118bcf31e11edb451b8aeed79c0cc,False
028dab46f31e11edb451b8aeed79c0cc,Freeway Killer,2010,Freeway Killer 2010 poster.jpg,"[br, Cole Williams, br, Dusty Sorg , br, Deb...",NaN,"[[0.3437575828245501, -0.31658492783599557, 0....",0.156828,97.8,2.0,"[mystery, conspiracy, suspense]",028dab46f31e11edb451b8aeed79c0cc,False
0b3c5ca6f31e11edb451b8aeed79c0cc,Bad Girl,2016,Bad Girl (2016 film) poster.png,"[Sara West, Samara Weaving, Ben Winspear\n, F...",NaN,"[[0.2675615539151958, 0.0710663661774484, -0.0...",0.168959,98.72,3.0,"[travel, adventure, journey, future]",0b3c5ca6f31e11edb451b8aeed79c0cc,False
10c32e16f31e11edb451b8aeed79c0cc,El Candidato,2016,El Candidato (2016 film) poster.jpg,"[br, Hernán Vidaurre, br, Guillermo Rossini, b...",NaN,"[[0.1196766110006551, 0.02902790634081226, -0....",0.181411,89.95,4.0,"[business, action, journey]",10c32e16f31e11edb451b8aeed79c0cc,False


In [ ]:
def loadJson(path):
    f = open(path)
    data = json.load(f)
    f.close()
    return data

def moving_average(x, width):
    return np.convolve(x, np.ones(width), 'valid') / width

def computeNormAvg(movie, avgWidth):
    neg = [entry[1] for entry in movie]
    pos = [entry[2] for entry in movie]
    avgNeg = moving_average(neg, avgWidth)
    avgPos = moving_average(pos, avgWidth)
    diff = []
    for i in range(len(avgPos)):
        diff.append(avgPos[i] - avgNeg[i])
    x = [entry[0] for entry in movie[:len(avgPos)]]
    factor = 100 / x[-1]
    normx = [r * factor for r in x]
    return normx, avgNeg, avgPos, diff

def plot(x, neg=None, pos=None, diff=None, poly=None):
    plt.figure(figsize=(8, 4), layout='constrained')
    if neg is not None: plt.plot(x, neg, color="red", label="neg")
    if pos is not None: plt.plot(x, pos, color="green", label="pos")
    if diff is not None: plt.plot(x, diff, color="blue", label="delta")
    if poly is not None: plt.plot(x, poly, color="green", label="fit")
    plt.axhline(y=0, color="black", linestyle='--')
    plt.xlabel("time [s]")
    plt.ylabel("sentiment")
    plt.legend()

In [60]:
# get the first row of the dataframe
rows = df.iloc[:2]


In [70]:
for l,s in rows.iterrows():
    n_path = f'{path}\\analysis\{s["uuid"]}.json'

In [71]:
analysis_data = loadJson(n_path)

In [ ]:
x, neg, pos, diff = computeNormAvg(analysis_data, 128)
fit1 = list(np.polyfit(x, diff, 16))
fit1.reverse()
func = [sum([val**i * fit1[i] for i in range(len(fit1))]) for val in x]
plot(x, diff=diff, poly=func)